## This is a web scrapping for raw data using BeautifulSoup

In [2]:
# import libraries
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

Set up global variables and Dataframe from pandas to store values that we are going to retrieve/scrap from and use for loop to repeat it

In [27]:
base_url = "https://sofifa.com/players?offset="
columns = ["ID", "Name", "Age", "Position", "Nationality", "Overall",
          "Potential", "Club", "Value", "Wage"]

data = pd.DataFrame(columns = columns)
for offset in range(0, ):
    url = base_url + str(offset * 60)
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text, "html.parser")
    table_body = soup.find("tbody")
    for row in table_body.findAll("tr"):
        td = row.findAll("td")
        pid = td[0].find("img").get("id")
        nationality = td[1].find("img").get("title")
        name = td[1].find("a").get("aria-label")
        rel = td[1].findAll("a", {"rel" : "nofollow"})
        pos = rel[0].findAll("span")
        for span in pos:
            positions= (span.text.split) ## FIX HERE
        age = td[2].text
        overall = td[3].text.strip()
        potential = td[4].text.strip()
        club = td[5].find("a").text
        value = td[6].text.strip()
        wage = td[7].text.strip()
        # store the retrieved 
        player_data = pd.DataFrame([[pid, name, age, positions,nationality, overall,potential, club,value,wage]])
        player_data.columns = columns
        data = pd.concat([data, player_data])
    print("done for "+str(offset), end = "\r")
data.drop_duplicates()
data.head()
data.to_csv('../data/player_data.csv', encoding='utf-8-sig')
    